In [ ]:
import sys
sys.path.extend([ '../', '../../', '../../../'])
import torch
import dataloader as dl
from args import Args
# from model import Het_ConEn, Het_NetEn, EdgePredictor, Het_classify
from model import MAG_ConEn, MAG_NetEn, EdgePredictor, MAG_classify
from train import train_smote

# Set device to GPU if available, else use CPU
args = Args()
args.imdb()
torch.cuda.empty_cache()

Current device: NVIDIA A100 80GB PCIe


In [2]:
data = torch.load('../../data/data.pt')
print(data)

HeteroData(
  m={
    num_nodes=4666,
    y=[4666],
  },
  a={ num_nodes=5845 },
  d={ num_nodes=2271 },
  m_text_embed={ x=[4666, 128] },
  m_net_embed={ x=[4666, 128] },
  m_a_net_embed={ x=[4666, 128] },
  m_d_net_embed={ x=[4666, 128] },
  a_net_embed={ x=[5845, 128] },
  a_text_embed={ x=[5845, 128] },
  d_net_embed={ x=[2271, 128] },
  d_text_embed={ x=[2271, 128] },
  (m, walk, m)={ edge_index=[2, 32862] },
  (m, walk, a)={ edge_index=[2, 29894] },
  (m, walk, d)={ edge_index=[2, 13656] },
  (a, walk, m)={ edge_index=[2, 26087] },
  (a, walk, a)={ edge_index=[2, 35045] },
  (a, walk, d)={ edge_index=[2, 15898] },
  (d, walk, m)={ edge_index=[2, 11136] },
  (d, walk, a)={ edge_index=[2, 14009] },
  (d, walk, d)={ edge_index=[2, 6256] }
)


In [3]:
print(data['m','walk','a'].edge_index)
device = args.device
data = data.to(device)

# Send all x tensors to the device
data['m_text_embed']['x'] = data['m_text_embed']['x'].to(device)
data['m_net_embed']['x'] = data['m_net_embed']['x'].to(device)
data['m_a_net_embed']['x'] = data['m_a_net_embed']['x'].to(device)
data['m_d_net_embed']['x'] = data['m_d_net_embed']['x'].to(device)
data['a_net_embed']['x'] = data['a_net_embed']['x'].to(device)
data['a_text_embed']['x'] = data['a_text_embed']['x'].to(device)
data['d_net_embed']['x'] = data['d_net_embed']['x'].to(device)
data['d_text_embed']['x'] = data['d_text_embed']['x'].to(device)

# Send all y tensors to the device
data['m']['y'] = data['m']['y'].to(device)

# Send all edge_index tensors to the device
data['m', 'walk', 'm']['edge_index'] = data['m', 'walk', 'm']['edge_index'].to(device)
data['m', 'walk', 'a']['edge_index'] = data['m', 'walk', 'a']['edge_index'].to(device)
data['m', 'walk', 'd']['edge_index'] = data['m', 'walk', 'd']['edge_index'].to(device)
data['a', 'walk', 'm']['edge_index'] = data['a', 'walk', 'm']['edge_index'].to(device)
data['a', 'walk', 'a']['edge_index'] = data['a', 'walk', 'a']['edge_index'].to(device)
data['a', 'walk', 'd']['edge_index'] = data['a', 'walk', 'd']['edge_index'].to(device)
data['d', 'walk', 'm']['edge_index'] = data['d', 'walk', 'm']['edge_index'].to(device)
data['d', 'walk', 'a']['edge_index'] = data['d', 'walk', 'a']['edge_index'].to(device)
data['d', 'walk', 'd']['edge_index'] = data['d', 'walk', 'd']['edge_index'].to(device)

edge_indices = [ data['m', 'walk', 'm'].edge_index, data['m', 'walk', 'a'].edge_index, data['m', 'walk', 'd'].edge_index ]

tensor([[   0,    0,    0,  ..., 4665, 4665, 4665],
        [  87, 4074, 3760,  ...,  999, 2508, 2888]])


In [4]:
c_train_num = dl.train_num(data['m'].y, args.im_class_num, args.class_samp_num[0], args.im_ratio)
print(c_train_num, sum(c_train_num))
train_idx, val_idx, test_idx, c_num_mat = dl.segregate(data['m'].y, c_train_num, args.seed[1], args)
print("train_idx: ", train_idx, len(train_idx))
print("val_idx: ", val_idx, len(val_idx))
print("test_idx: ", test_idx, len(test_idx))
# print(c_num_mat)

[40, 60, 100] 200
0 543
1 924
2 1473
train_idx:  [2745, 1616, 1260, 3151, 1623, 3752, 1211, 991, 3326, 1341, 2098, 2911, 864, 1976, 2092, 911, 3144, 3024, 2899, 2443, 2621, 3905, 1525, 3056, 3411, 1128, 1915, 4342, 3010, 1233, 691, 389, 3457, 3981, 4367, 729, 1430, 202, 2629, 3077, 3623, 4208, 2009, 1448, 572, 3642, 3944, 952, 3990, 6, 1694, 4396, 4391, 833, 1512, 4183, 1609, 2804, 635, 1371, 1483, 2487, 762, 2436, 2706, 576, 4446, 3647, 646, 1533, 1816, 4366, 2124, 3415, 4448, 1232, 2525, 2063, 4198, 2321, 4092, 2091, 2093, 3084, 2850, 3839, 3297, 3096, 1870, 3333, 406, 3021, 797, 3554, 3614, 4, 3048, 2517, 1950, 4091, 2467, 564, 1642, 751, 3356, 3394, 1996, 2151, 190, 1499, 2507, 1645, 2594, 731, 556, 2662, 1582, 3874, 1099, 1413, 879, 451, 1486, 2585, 694, 2883, 1245, 2956, 1300, 165, 1076, 2674, 2383, 852, 2235, 275, 502, 4021, 2832, 384, 2081, 2178, 2367, 4553, 3250, 2267, 1011, 2120, 4118, 496, 531, 3668, 2644, 664, 3631, 3351, 1557, 1311, 2932, 2282, 4346, 1095, 4068, 3853, 1322

In [5]:
## HAN

encoder1 = MAG_ConEn(args.embed_dim, args.dropout)
encoder2 = MAG_NetEn(args.embed_dim, args.dropout)
classifier = MAG_classify(args.embed_dim, args.nclass, args.dropout)
decoder_m = EdgePredictor(args.embed_dim)
decoder_a = EdgePredictor(args.embed_dim)
decoder_d = EdgePredictor(args.embed_dim)
# decoder_m = torch.load('pretrained/decoder_m.pth')
# decoder_a = torch.load('pretrained/decoder_a.pth')
# decoder_d = torch.load('pretrained/decoder_d.pth')
decoder_list = [decoder_m, decoder_a, decoder_d]

encoder1.to(device)
encoder2.to(device)
classifier.to(device)
for decoder in decoder_list:
    decoder.to(device)

In [6]:
### HETGNN

# encoder1 = Het_ConEn(args.embed_dim, args.dropout)
# encoder2 = Het_NetEn(args.embed_dim, args.dropout)
# classifier = Het_classify(args.embed_dim, args.nclass, args.dropout)
# decoder_m = EdgePredictor(args.embed_dim)
# decoder_a = EdgePredictor(args.embed_dim)
# decoder_d = EdgePredictor(args.embed_dim)
# # decoder_m = torch.load('pretrained/decoder_m.pth')
# # decoder_a = torch.load('pretrained/decoder_a.pth')
# # decoder_d = torch.load('pretrained/decoder_d.pth')
# decoder_list = [decoder_m, decoder_a, decoder_d]

# encoder1.to(device)
# encoder2.to(device)
# classifier.to(device)
# for decoder in decoder_list:
#     decoder.to(device)

## Training Part

In [7]:
torch.cuda.empty_cache()
train_smote(data, edge_indices, encoder1, encoder2, classifier, decoder_list, 
            train_idx, val_idx, test_idx, args, os_mode = 'gsm', train_mode = 'recon')

[tensor(0.0008, device='cuda:0', grad_fn=<DivBackward0>)]
[tensor(0.0008, device='cuda:0', grad_fn=<DivBackward0>), tensor(0.0008, device='cuda:0', grad_fn=<DivBackward0>)]
[tensor(0.0008, device='cuda:0', grad_fn=<DivBackward0>), tensor(0.0008, device='cuda:0', grad_fn=<DivBackward0>), tensor(0.0002, device='cuda:0', grad_fn=<DivBackward0>)]
Epoch [1/200], Loss: 0.5029, Accuracy: 0.3267, Edge Accuracy: [0.9984905993857306, 0.9989038883839082, 0.9987112708873489]
class 0: 0.1, Class 0: AUC-ROC- 0.4897, F1 Score- 0.5000; class 1: 0.3448979591836735, Class 1: AUC-ROC- 0.5030, F1 Score- 0.5025; class 2: 0.1, Class 2: AUC-ROC- 0.4336, F1 Score- 0.5000; Macro-Average AUC-ROC: 0.4754; Macro-Average F1 Score: 0.5008
Validation Loss: 0.0000, Validation Accuracy: 0.3333, Validation Edge Accuracy: []
class 0: 0.1, Class 0: AUC-ROC- 0.5029, F1 Score- 0.5000; class 1: 0.1, Class 1: AUC-ROC- 0.5943, F1 Score- 0.5000; class 2: 0.1, Class 2: AUC-ROC- 0.4640, F1 Score- 0.5000; Macro-Average AUC-ROC: 0

([0.3333333333333333,
  0.3333333333333333,
  0.3333333333333333,
  0.3333333333333333,
  0.3333333333333333,
  0.3333333333333333,
  0.3333333333333333,
  0.3333333333333333,
  0.3333333333333333,
  0.3333333333333333,
  0.3333333333333333,
  0.3333333333333333,
  0.3333333333333333,
  0.3333333333333333,
  0.3333333333333333,
  0.3333333333333333,
  0.3333333333333333,
  0.3333333333333333,
  0.3333333333333333,
  0.3333333333333333,
  0.3333333333333333],
 [0.5305703703703704,
  0.5338425925925926,
  0.5371388888888889,
  0.5354685185185185,
  0.534712962962963,
  0.5332574074074073,
  0.5284712962962963,
  0.5227046296296297,
  0.5164111111111112,
  0.5095861111111111,
  0.5017944444444444,
  0.49586388888888894,
  0.49267129629629625,
  0.49025,
  0.48726018518518516,
  0.48433518518518515,
  0.4837037037037037,
  0.4813018518518519,
  0.4824435185185185,
  0.48639537037037034,
  0.5372537037037037],
 [0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.

In [8]:
# test_smote(data, edge_indices, encoder, classifier, decoder_list, test_idx, args = args, dataset = 'Test', os_mode = 'gsm', train_mode = 'preT')

In [ ]:
# torch.save(decoder_list[0], '../pretrained_magnn/decoder_m.pth')
# torch.save(decoder_list[1], '../pretrained_magnn/decoder_a.pth')
# torch.save(decoder_list[2], '../pretrained_magnn/decoder_d.pth')
# torch.save(encoder1, '../pretrained_magnn/encoder1.pth')
# torch.save(encoder2, '../pretrained_magnn/encoder2.pth')